In [2]:
import tkinter as tk
from tkinter import messagebox

def check_length(event):
    user_input = entry.get()
    if len(user_input) != 10:
        messagebox.showwarning("Warning", "Input must be 10 digits.")

root = tk.Tk()
root.title("Input Length Checker")

label = tk.Label(root, text="Enter 10-digit input:")
label.pack(pady=5)

entry = tk.Entry(root)
entry2 = tk.Entry(root)
entry.pack(pady=5)
entry2.pack(pady=10)
entry.bind("<FocusOut>", check_length)  # Bind event listener to check length when focus leaves the entry field

root.mainloop()


In [1]:
import sqlite3
import pandas as pd
#import pandas as pd
import gspread as gs
from google.oauth2 import service_account
import time
import datetime
#from gspread_pandas import Spread, Client
start_time = time.time()
SCOPES = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]
SERVICE_ACCOUNT_FILE = r'C:\Users\KP\Development\IMS_2024\main_ttk\sheets-to-python-credential.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

client = gs.authorize(credentials)

Spread = client.open("OP Register")


pharmacyWS = Spread.worksheet("Pharmacy")
def pharmData():
        pwsFirstRow = pharmacyWS.row_values(1)
        print(pwsFirstRow)
        pwsBillNoColNo = pwsFirstRow.index("Bill No")+1
        pwsMedNameColNo = pwsFirstRow.index("Medicine name")+1
        pwsDateColNo = pwsFirstRow.index("Date")+1
        pwsPatientNameColNo = pwsFirstRow.index("Name")+1
        pwsQtyColNo = pwsFirstRow.index("Quantity")+1
        

        pwsLastRowNo = pharmacyWS.find("", in_column  = pwsDateColNo).row
        #print(pwsLastRowNo, pwsBillNoColNo, pwsMedNameColNo, pwsDateColNo, pwsQtyColNo, pwsPatientNameColNo)
        return pwsLastRowNo, pwsBillNoColNo, pwsMedNameColNo, pwsDateColNo, pwsQtyColNo, pwsPatientNameColNo

"""start_time = time.time()

pharmData()
end_time = time.time()

print(end_time-start_time)"""

medListWS = Spread.worksheet("Medicine List")


def getMedData():
        mlsMedNameColNo = medListWS.find("Name", in_row=1).col
        medList = medListWS.col_values(mlsMedNameColNo)
        medList = medList[2:]
        medList.sort(key=str.lower)
        return medList








list_of_lists = medListWS.get_all_values()
medListData = pd.DataFrame(list_of_lists)
medListData.columns = medListData.iloc[0]
medListData = medListData[1:]
#print(medListData)



def getInvoiceDate():
        insFirstRow = inoviceWS.row_values(1)
        insinvNoColNo= insFirstRow.index("Inovice No")+1
        insPatientNameColNo= insFirstRow.index("Name")+1
        insBillAmountColNo= insFirstRow.index("Bill Amount")+1
        insDiscountColNo= insFirstRow.index("Discount")+1
        insPayModeColNo= insFirstRow.index("Payment Mode")+1
        insCashColNo= insFirstRow.index("Cash")+1
        insUPIColNo= insFirstRow.index("UPI")+1
        insDateColNo = insFirstRow.index("Date")+1
        insLastRowNo = len(inoviceWS.col_values(insDateColNo))+1
        return insLastRowNo, insPatientNameColNo, insDateColNo, insBillAmountColNo, insPayModeColNo, insDiscountColNo, insCashColNo, insUPIColNo, insinvNoColNo


inoviceWS = Spread.worksheet("Invoices")

opWS =Spread.worksheet("OP")

def getOPData():
        oPFirstRow = opWS.row_values(1)
        oPUIDColNo = oPFirstRow.index("UID")+1
        oPDateColNo = oPFirstRow.index("Date")+1
        oPNameColNo = oPFirstRow.index("Name")+1
        oPPhoneColNo = oPFirstRow.index("Phone No")+1
        oPGenderColNo = oPFirstRow.index("Gender")+1
        oPAgeColNo = oPFirstRow.index("Age")+1
        oPPayModeColNo = oPFirstRow.index("Payment mode")+1
        oPAmountColNo = oPFirstRow.index("Amount")+1
        opLastRow = len(opWS.col_values(oPNameColNo))

        return oPUIDColNo, oPDateColNo, oPNameColNo, oPPhoneColNo, oPPayModeColNo, oPAmountColNo, opLastRow, oPGenderColNo, oPAgeColNo


aCountWS = Spread.worksheet("ACount")

def getClientid(name):
    name = name.strip()
    name = name.replace(" ", "")
    name = name.replace(".", "")
    now = datetime.datetime.now()  
    year_str = str(now.year)[-2:]  
    month_str = str(now.month).zfill(2)
    name_prefix = name[:3].upper()
    first_letter = name_prefix[0]   
    
    currLetter = aCountWS.find(first_letter,in_column=1).row
    currCount = int(aCountWS.cell(currLetter,2).value)

    
    count = currCount + 1
    serial_num = str(count).zfill(2)
    
    return f"{year_str}{month_str}{name_prefix}{serial_num}"





def getBillNo():
        insDateColNo = inoviceWS.find("Date", in_row=1).col
        insinvNoColNo = inoviceWS.find("Inovice No", in_row=1).col
        insLastRowNo = len(inoviceWS.col_values(insDateColNo))

        currBillNo = inoviceWS.cell(insLastRowNo, insinvNoColNo).value

        return currBillNo

end_time = time.time()

print(end_time-start_time)



4.396560192108154


In [6]:
for x in range (407,473):
    name = opWS.cell(x,11).value
    cid = getClientid(name)
    opWS.update_cell(x,3,name)
    opWS.update_cell(x,1,cid)
    opWS.update_cell(x,11,"")
    time.sleep(1)

In [9]:
#def getCurrentDayPatients():
from datetime import date

today = date.today().strftime("%d-%b")
columnheaders = opWS.row_values(1)
oPUIDColNo, oPDateColNo, oPNameColNo, oPPhoneColNo, oPPayModeColNo, oPAmountColNo, opLastRow, oPGenderColNo, oPAgeColNo = getOPData()
rowsWithDate = [opWS.row_values(x.row) for x in opWS.findall(today, in_column=oPDateColNo)]
currentDayPatients = pd.DataFrame(rowsWithDate)
print(columnheaders)
currentDayPatients
currentDayPatients.columns = columnheaders
currentDayPatients    
    #$return currentDayPatients

#getCurrentDayPatients()

['UID', 'Date', 'Name', 'Phone No', 'Gender', 'Age', 'Procedure', 'Payment mode', 'Amount', 'Comments']


,UID,Date,Name,Phone No,Gender,Age,Procedure,Payment mode,Amount,Comments
0,2402HUS96,27-Feb,S.Saraswati Amma,9963148667,,,NO,,NO,None
1,2402HUS97,27-Feb,S.Jayala Lakshmi,9441088440,,,NO,,NO,None
2,2402HUS98,27-Feb,Asma,8074640549,,,con,Cash,200,None
3,2402HUS99,27-Feb,Sartaj,9885385417,,,con,Cash,200,CLEAR
4,2402HUS100,27-Feb,Aryan,9154762075,,,con,Cash,200,None
